# Market Basket Analysis

In [1]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [90]:
df_original = pd.read_excel('dummy original.xlsx', sheet_name='MARKET BASKET')

In [92]:
df = df_original

In [93]:
df.head(10)

,Obs,Transaction_ID,CARDNUM,Amount,TIMESTAMP,ISSR_BUS_NM,prch_dt,CPD_Month,_NAME_,PROD1,...,PROD50,PROD51,PROD52,PROD53,PROD54,PROD55,PROD56,PROD57,PROD58,PROD59
0,1,TM1,3,320000,2019-05-28 15:13:23.002,A,20190528,201905,Product_Group_Name___Level_4,1504:\nCHEESE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,TM2,6,81000,2019-04-17 16:30:48.038,B,20190419,201904,Product_Group_Name___Level_4,1424:\nCHOCOLATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,TM3,2,226804,2019-04-26 20:05:33.043,C,20190330,201904,Product_Group_Name___Level_4,1222: Sanitary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,TM4,5,270800,2019-03-23 16:19:13.987,C,20190309,201903,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,TM5,10,514080,2019-03-24 13:02:16.022,D,20190511,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,TM6,8,258840,2019-04-14 14:24:06.998,C,20180728,201807,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,TM7,3,258840,2019-04-14 14:24:06.998,B,20180914,201809,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,TM8,8,517680,2019-04-17 10:18:16.013,C,20190501,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,TM9,8,243720,2019-04-17 15:19:49.037,A,20190430,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,TM10,1,243720,2019-04-17 15:19:49.037,C,20190523,201905,Product_Group_Name___Level_4,1440: BABY\nMILK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df.shape

(100, 68)

In [95]:
# simpan nama2 kolom kedalam variabel column_name
column_name = df.columns

In [96]:
column_name

Index(['Obs', 'Transaction_ID', 'CARDNUM', 'Amount', 'TIMESTAMP',
       'ISSR_BUS_NM', 'prch_dt', 'CPD_Month', '_NAME_', 'PROD1', 'PROD2',
       'PROD3', 'PROD4', 'PROD5', 'PROD6', 'PROD7', 'PROD8', 'PROD9', 'PROD10',
       'PROD11', 'PROD12', 'PROD13', 'PROD14', 'PROD15', 'PROD16', 'PROD17',
       'PROD18', 'PROD19', 'PROD20', 'PROD21', 'PROD22', 'PROD23', 'PROD24',
       'PROD25', 'PROD26', 'PROD27', 'PROD28', 'PROD29', 'PROD30', 'PROD31',
       'PROD32', 'PROD33', 'PROD34', 'PROD35', 'PROD36', 'PROD37', 'PROD38',
       'PROD39', 'PROD40', 'PROD41', 'PROD42', 'PROD43', 'PROD44', 'PROD45',
       'PROD46', 'PROD47', 'PROD48', 'PROD49', 'PROD50', 'PROD51', 'PROD52',
       'PROD53', 'PROD54', 'PROD55', 'PROD56', 'PROD57', 'PROD58', 'PROD59'],
      dtype='object')

In [97]:
# tentukan kolom mana saja yg ingin dihapus
column_drop = ['Obs', 'Transaction_ID', 'Amount', 'TIMESTAMP',
       'ISSR_BUS_NM', 'prch_dt', 'CPD_Month', '_NAME_']

In [98]:
# hapus kolom yang tidak dibutuhkan 
df.drop(column_drop, axis=1, inplace=True)

In [99]:
df.head(10)

,CARDNUM,PROD1,PROD2,PROD3,PROD4,PROD5,PROD6,PROD7,PROD8,PROD9,...,PROD50,PROD51,PROD52,PROD53,PROD54,PROD55,PROD56,PROD57,PROD58,PROD59
0,3,1504:\nCHEESE,2201:\nSAYUR TAN,1003: FRUIT\nJUI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,1424:\nCHOCOLATE,1008:\nWATER,1500: FRESH\nMIL,1453:\nINSTANT D,1421:\nSOFT\nCAND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1222: Sanitary,1502: DAIRY\nDES,1414: ROLLS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,1440: BABY\nMILK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
# ambil atau load data result_predict dari csv file
df_result_predict = pd.read_csv('result_predict.csv')

In [101]:
df_result_predict.head(10)

,cardno,train,test
0,1,NaN,5
1,2,NaN,1
2,3,NaN,1
3,4,NaN,4
4,5,NaN,1
5,6,NaN,1
6,7,1.0,1
7,8,1.0,1
8,9,1.0,1
9,10,NaN,1


In [102]:
# dapatkan data cardnum dari df dan cardno dari df_result_predict, lalu ubah tipe data nya menjadi list
cardnum = df['CARDNUM'].values.reshape(-1).tolist()
cardno = df_result_predict['cardno'].values.reshape(-1).tolist()

In [103]:
# dapatkan nilai cluster pada df sesuai cluster pada df_result_predict
clusters = []
for index1, val1 in enumerate(cardnum):
    found = False
    for index2, val2 in enumerate(cardno):
        if(val1 == val2):
            found = True
            clusters.append(df_result_predict.ix[index2, 'test'])
            break
    if(found == False):
        clusters.append(None)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [104]:
print(clusters)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 5, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 5, 1, 5, 5, 1, 1, 1, 5]


In [117]:
# hitung jumlah data permasing2 cluster
unique_elements, counts_elements = np.unique(clusters, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[ 1  4  5]
 [78  9 13]]


In [105]:
# lengketkan nilai cluster ke df
df['cluster'] = clusters

In [106]:
# simpan df dengan cluster ke dalam file csv
df.to_csv('mba_cluster.csv', index=False)

In [ ]:
# grouping cluster
# pisahkan data pada df sesuai dengan cluster yang telah ditentukan di df_result_predict
# lalu simpan ke dalam csv file sesuai dengan masing2 cluster
for i in range(0, 10):
    if(i in clusters):
        df_cluster = pd.DataFrame(columns=df.columns)
        for x, value in enumerate(clusters):
            if(value == i):
                df_cluster = df_cluster.append(df.iloc[x], ignore_index = True)

        df_cluster.to_csv('cluster_'+str(i)+'.csv', index=False)